In [2]:
from google.colab import drive
import pandas as pd 
import torch
import torch.nn.functional as F
import torchvision
from torchvision.datasets import ImageFolder
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as T
import numpy as np
import os
import shutil
from PIL import Image
!pip install efficientnet_pytorch
from efficientnet_pytorch import EfficientNet
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler

%matplotlib inline


  Created wheel for efficientnet-pytorch: filename=efficientnet_pytorch-0.7.1-py3-none-any.whl size=16446 sha256=fa52180a3547ee43a15dcd795fcd420d05128cab269c3acedd442fbad0643c8d
  Stored in directory: /root/.cache/pip/wheels/0e/cc/b2/49e74588263573ff778da58cc99b9c6349b496636a7e165be6
Successfully built efficientnet-pytorch


In [1]:
drive.mount('/content/gdrive')  # link google drive


NameError: ignored

In [ ]:
label_header = ["photo", "type"]
labels = pd.read_csv("/content/gdrive/My Drive/VRDL_HW1/training_labels.txt",
                     names=label_header, sep=' ')
train_fold = "/content/gdrive/MyDrive/VRDL_HW1/train_images_new"
valid_fold = "/content/gdrive/MyDrive/VRDL_HW1/validation_images"

class_header = ["type"]
class_data = pd.read_csv("/content/gdrive/My Drive/VRDL_HW1/classes.txt",
                         names=class_header)
test_fold = "/content/gdrive/MyDrive/VRDL_HW1/test_images"


                           type
0    001.Black_footed_Albatross
1          002.Laysan_Albatross
2           003.Sooty_Albatross
3         004.Groove_billed_Ani
4            005.Crested_Auklet
..                          ...
195              196.House_Wren
196              197.Marsh_Wren
197               198.Rock_Wren
198             199.Winter_Wren
199     200.Common_Yellowthroat

[200 rows x 1 columns]
001.Black_footed_Albatross


In [ ]:
# make training data to different folder
# foldername = "/content/gdrive/MyDrive/VRDL_HW1/train_images"
# for index in range(len(labels['photo'])):
#     print(data)
#     print(labels['photo'][index], labels['type'][index])
#     fdname = train_fold + '/' + labels['type'][index]
#     os.makedirs(fdname, exist_ok=True)
#     shutil.move(foldername + '/' + labels['photo'][index], fdname)


In [ ]:
# divide train data to validation data
# for index in range(1):
#     fdname_old = train_fold + '/' + class_data['type'][index]
#     fdname_new = valid_fold + '/' + class_data['type'][index]
#     os.makedirs(fdname_new, exist_ok=True)
#     j = 0
#     print(fdname_old)
#     for file in os.listdir(fdname_old):
#         if(j < 1):
#             shutil.move(fdname_old + '/' + file, fdname_new + '/' + file)
#             j += 1


In [ ]:
imagenet_stats = ([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])

# adjust to bigger,random get 350*350,
# do other change to get more data,then normalize
train_transforms = T.Compose([
    T.Resize((375, 375)),
    T.RandomCrop((350, 350)),
    T.RandomHorizontalFlip(),
    T.RandomRotation(10),
    T.ToTensor(),
    T.Normalize(*imagenet_stats, inplace=True),
    T.RandomErasing(inplace=True)
])

valid_transforms = T.Compose([
    T.Resize((350, 350)),
    T.ToTensor(),
    T.Normalize(*imagenet_stats)
])

test_transforms = T.Compose([
    T.Resize((350, 350)),
    T.ToTensor(),
    T.Normalize(*imagenet_stats)
])


In [ ]:
# get train/valid dataset
train_dataset = ImageFolder(train_fold, train_transforms)
valid_dataset = ImageFolder(valid_fold, valid_transforms)
# get model
model = EfficientNet.from_pretrained('efficientnet-b6')


Downloading: "https://github.com/lukemelas/EfficientNet-PyTorch/releases/download/1.0/efficientnet-b6-c76e70fd.pth" to /root/.cache/torch/hub/checkpoints/efficientnet-b6-c76e70fd.pth


  0%|          | 0.00/165M [00:00<?, ?B/s]

Loaded pretrained weights for efficientnet-b6


In [ ]:
# to see model's fc layer
model


In [ ]:
model.fc = torch.nn.Linear(2304, 200)

# use gpu
model = model.cuda()
try:
    model.load_state_dict(torch.load(
        '/content/gdrive/MyDrive/VRDL_HW1/weight_eff.pth'))
    model.eval()
    print('have previous weight')
except:
    print('Cannot load previous weight')

loss_fn = nn.CrossEntropyLoss()
opt = optim.Adam(model.parameters(), 1e-4)
scheduler = lr_scheduler.ReduceLROnPlateau(
            opt, mode='max', factor=0.1, patience=1,
            verbose=True, threshold=0.0001, threshold_mode='rel',
            cooldown=0, min_lr=0, eps=1e-08)


have previous weight


In [ ]:
torch.cuda.empty_cache()
# Create train/valid loaders
batch_size = 8
train_dataloader = DataLoader(train_dataset, batch_size,
                              shuffle=True, num_workers=2, pin_memory=True)
valid_dataloader = DataLoader(valid_dataset, batch_size,
                              shuffle=False, num_workers=2, pin_memory=True)


In [ ]:
valid_loss_min = np.Inf
n = 15

for epoch in range(n):
    # keep track of training and validation loss
    train_loss = 0.0
    valid_loss = 0.0
    print('running epoch: {}'.format(epoch))

    # train the model
    model.train()
    train_acc = 0.0
    valid_acc = 0.0
    for data, label in train_dataloader:
        torch.cuda.empty_cache()
        # move tensors to GPU if CUDA is available
        data = data.cuda()
        label = label.cuda()
        # clear the gradients of all optimized variables
        opt.zero_grad()
        # forward pass: compute predicted outputs
        # by passing inputs to the model
        pred = model(data)
        pred_label = torch.argmax(pred, dim=1)

        # calculate the batch loss
        loss = loss_fn(pred, label)
        # backward pass: compute gradient of the loss
        # with respect to model parameters
        loss.backward()
        # update training loss
        train_loss += loss.item()*data.size(0)
        train_acc += np.sum(pred_label.cpu().numpy() == label.cpu().numpy())
        # change learning rate to loss
        scheduler.step(train_acc)
        # perform a single optimization step (parameter update)
        opt.step()

    # validate the model
    model.eval()
    for data, label in valid_dataloader:
        data = data.cuda()
        label = label.cuda()
        opt.zero_grad()
        pred = model(data)
        pred_label = torch.argmax(pred, dim=1)
        loss = loss_fn(pred, label)
        loss.backward()
        valid_loss += loss.item()*data.size(0)
        valid_acc += np.sum(pred_label.cpu().numpy() == label.cpu().numpy())
        opt.step()

    # calculate average losses
    train_loss = train_loss/len(train_dataloader.dataset)
    valid_loss = valid_loss/len(valid_dataloader.dataset)
    train_acc = train_acc/len(train_dataloader.dataset)
    valid_acc = valid_acc/len(valid_dataloader.dataset)

    # print training/validation statistics
    print(f'\tTraining Loss: {train_loss:.6f}')
    print(f'\tValidation Loss: {valid_loss:.6f}')
    print(f'\t Training acc: {train_acc:.6f}')
    print(f'\tValidation acc: {valid_acc:.6f}')

    if valid_loss <= valid_loss_min:
        torch.save(model.state_dict(),
                   '/content/gdrive/MyDrive/VRDL_HW1/weight_eff.pth')
        print('Validation loss decreased ({:.6f} --> {:.6f}).Saving model ...'
              .format(valid_loss_min, valid_loss))


running epoch: 0
	Training Loss: 0.383638 	Validation Loss: 0.026924 	 Training acc: 0.916429 	Validation acc: 0.995000
Validation loss decreased (inf --> 0.026924).  Saving model ...
running epoch: 1
Epoch   352: reducing learning rate of group 0 to 1.0000e-05.
Epoch   354: reducing learning rate of group 0 to 1.0000e-06.
Epoch   356: reducing learning rate of group 0 to 1.0000e-07.
Epoch   358: reducing learning rate of group 0 to 1.0000e-08.
	Training Loss: 0.336117 	Validation Loss: 0.002722 	 Training acc: 0.927143 	Validation acc: 1.000000
Validation loss decreased (inf --> 0.002722).  Saving model ...
running epoch: 2
	Training Loss: 0.359398 	Validation Loss: 0.002663 	 Training acc: 0.922500 	Validation acc: 1.000000
Validation loss decreased (inf --> 0.002663).  Saving model ...
running epoch: 3
	Training Loss: 0.349662 	Validation Loss: 0.002786 	 Training acc: 0.924286 	Validation acc: 1.000000
Validation loss decreased (inf --> 0.002786).  Saving model ...
running epoch: 4

In [ ]:
img_order_header = ["name"]
order = pd.read_csv(
        "/content/gdrive/My Drive/VRDL_HW1/testing_img_order.txt",
        names=img_order_header)


3033


In [ ]:
model.eval()
submission = []
for i in range(len(order)):  # image order is important to your result
    filename = order['name'][i]
    img = Image.open(test_fold + '/' + filename)
    img = test_transforms(img)
    img = torch.unsqueeze(img, 0)
    img = img.cuda()
    pred = model(img)  # the predicted category
    pred_label = torch.argmax(pred, dim=1)
    idx = pred_label[0].item()
    print(filename, train_dataset.classes[idx])
    tmp = [filename, train_dataset.classes[idx]]
    submission.append(tmp)

np.savetxt('/content/gdrive/My Drive/VRDL_HW1/answer.txt',
           submission, fmt='%s')


串流輸出內容已截斷至最後 5000 行。
171
1953.jpg 172.Nashville_Warbler
65
1358.jpg 066.Western_Gull
152
4924.jpg 153.Philadelphia_Vireo
169
5942.jpg 170.Mourning_Warbler
28
2042.jpg 029.American_Crow
44
3852.jpg 045.Northern_Fulmar
53
4457.jpg 054.Blue_Grosbeak
131
0234.jpg 132.White_crowned_Sparrow
138
3916.jpg 139.Scarlet_Tanager
140
3026.jpg 141.Artic_Tern
19
3027.jpg 020.Yellow_breasted_Chat
1
3365.jpg 002.Laysan_Albatross
115
4094.jpg 116.Chipping_Sparrow
69
1340.jpg 070.Green_Violetear
59
4638.jpg 060.Glaucous_winged_Gull
60
5088.jpg 061.Heermann_Gull
0
2146.jpg 001.Black_footed_Albatross
59
0421.jpg 060.Glaucous_winged_Gull
6
2241.jpg 007.Parakeet_Auklet
106
3190.jpg 107.Common_Raven
44
5553.jpg 045.Northern_Fulmar
112
0827.jpg 113.Baird_Sparrow
87
1462.jpg 088.Western_Meadowlark
105
3950.jpg 106.Horned_Puffin
19
5794.jpg 020.Yellow_breasted_Chat
148
0348.jpg 149.Brown_Thrasher
100
2578.jpg 101.White_Pelican
46
3410.jpg 047.American_Goldfinch
32
1730.jpg 033.Yellow_billed_Cuckoo
66
3678.jpg 06